<a href="https://colab.research.google.com/github/phattaraphol/Football-Knowledge-RAG-System/blob/main/Football_Knowledge_RAG_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install all dependencies

In [ ]:
!pip install neo4j

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.4/325.4 kB 5.7 MB/s eta 0:00:00


In [ ]:
!pip install langchain-neo4j

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.2/313.2 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.8/204.8 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.5/329.5 kB 22.8 MB/s eta 0:00:00
  Attempting uninstall: neo4j
    Found existing installation: neo4j 6.0.3
    Uninstalling neo4j-6.0.3:
      Successfully uninstalled neo4j-6.0.3
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.12.0 which is incompatible.


In [ ]:
#ถอนการติดตั้งเพื่อไม่ให้ cache ใน colab กับ langchain เวอร์ชั่นเก่าเอาไว้
!pip uninstall -y langchain langchain-community langchain-core langchain-google-genai langchain-huggingface

#ติดตั้งใหม่โดย "ไม่ใช้ Cache" (--no-cache-dir) และ "ระบุเวอร์ชันล่าสุดที่เสถียร"
#จากนั้นตัว colab จะให้เรา restart run-time
!pip install --no-cache-dir \
    langchain==0.3.7 \
    langchain-community==0.3.7 \
    langchain-core==0.3.19 \
    langchain-google-genai \
    langchain-huggingface \
    faiss-cpu


Found existing installation: langchain 1.1.0
Uninstalling langchain-1.1.0:
  Successfully uninstalled langchain-1.1.0
Found existing installation: langchain-core 1.1.0
Uninstalling langchain-core-1.1.0:
  Successfully uninstalled langchain-core-1.1.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 6.3 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of langchain-google-genai to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of langchain-google-genai to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
INFO: pip is looking at multiple versions of langchain-huggingface to determine which version is compat

In [ ]:
import langchain
print(f"LangChain Version: {langchain.__version__}") #ต้องขึ้น 0.3.7 ไม่งั้นจะ import RetrievalQA ไม่ได้

#ลอง import
from langchain.chains import RetrievalQA
print("Import Success!")

LangChain Version: 0.3.7
Import Success!


## Prepare data

In [ ]:
from neo4j import GraphDatabase

uri = "neo4j+s://3581f28e.databases.neo4j.io"
user = "neo4j"
password = "pOtJfLTvttU6Dl4Vu5AxnwHeuY-ZatBWdhP4A_9kBjo"

driver = GraphDatabase.driver(uri, auth=(user, password))


In [ ]:
import pandas as pd

df = pd.read_csv("https://gist.githubusercontent.com/monkub003/1c6e1278930e65946e9b42ba869f453c/raw/64d6e21b974d5438c348dbf746b8b1a332e2f355/Football.csv")
df.head()

,Subject,Predicate,Object
0,Manchester United,playsIn,Premier League
1,Liverpool,playsIn,Premier League
2,Arsenal,playsIn,Premier League
3,Manchester City,playsIn,Premier League
4,Chelsea,playsIn,Premier League


Push data into Neo4j (Already pushed, shouldn't be repeat)

In [ ]:
# def push_spo(tx, s, p, o):
#     query = """
#     MERGE (sub:Entity {name: $s})
#     MERGE (obj:Entity {name: $o})
#     MERGE (sub)-[r:`%s`]->(obj)
#     """ % p.replace(" ", "_")
#     tx.run(query, s=s, o=o)

# with driver.session() as session:
#     for index, row in df.iterrows():
#         s = str(row["Subject"])
#         p = str(row["Predicate"])
#         o = str(row["Object"])

#         if pd.isna(s) or pd.isna(p) or pd.isna(o):
#             continue

#         session.execute_write(push_spo, s, p, o)

# print("Data pushed to Neo4j successfully!")


Data pushed to Neo4j successfully!


In [ ]:
with driver.session() as session:
    result = session.run("MATCH (n)-[r]->(m) RETURN n,r,m LIMIT 25")
    for record in result:
        print(record)


<Record n=<Node element_id='4:6cd6de29-4d49-4f07-bc5b-89f3f382d919:2' labels=frozenset({'Entity'}) properties={'name': 'Espanyol'}> r=<Relationship element_id='5:6cd6de29-4d49-4f07-bc5b-89f3f382d919:1152941295816146946' nodes=(<Node element_id='4:6cd6de29-4d49-4f07-bc5b-89f3f382d919:2' labels=frozenset({'Entity'}) properties={'name': 'Espanyol'}>, <Node element_id='4:6cd6de29-4d49-4f07-bc5b-89f3f382d919:25' labels=frozenset({'Entity'}) properties={'name': 'Barcelona'}>) type='cityIs' properties={}> m=<Node element_id='4:6cd6de29-4d49-4f07-bc5b-89f3f382d919:25' labels=frozenset({'Entity'}) properties={'name': 'Barcelona'}>>
<Record n=<Node element_id='4:6cd6de29-4d49-4f07-bc5b-89f3f382d919:4' labels=frozenset({'Entity'}) properties={'name': 'Manchester United'}> r=<Relationship element_id='5:6cd6de29-4d49-4f07-bc5b-89f3f382d919:1152925902653358084' nodes=(<Node element_id='4:6cd6de29-4d49-4f07-bc5b-89f3f382d919:4' labels=frozenset({'Entity'}) properties={'name': 'Manchester United'}>, <

# RAG

In [ ]:
import os
import pandas as pd

from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from neo4j import GraphDatabase
import difflib

from IPython.display import display, Markdown

In [1]:
google_API_key = "API_KEY"

สำหรับเช็ค Available model

In [ ]:
import google.generativeai as genai

genai.configure(api_key=google_API_key)

print("Available Models:")
for m in genai.list_models():
    if 'generateContent' in m.supported_generation_methods:
        print(m.name)

Available Models:
models/gemini-2.5-flash
models/gemini-2.5-pro
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-flash-lite-preview
models/gemini-2.0-pro-exp
models/gemini-2.0-pro-exp-02-05
models/gemini-exp-1206
models/gemini-2.5-flash-preview-tts
models/gemini-2.5-pro-preview-tts
models/learnlm-2.0-flash-experimental
models/gemma-3-1b-it
models/gemma-3-4b-it
models/gemma-3-12b-it
models/gemma-3-27b-it
models/gemma-3n-e4b-it
models/gemma-3n-e2b-it
models/gemini-flash-latest
models/gemini-flash-lite-latest
models/gemini-pro-latest
models/gemini-2.5-flash-lite
models/gemini-2.5-flash-image-preview
models/gemini-2.5-flash-image
models/gemini-2.5-flash-preview-09-2025
models/gemini-2.5-flash-lite-preview-09-2025
models/gemini-3-pro-preview
models/gemini-3-pro-image-preview
models/nano-banana-p

ตั้งค่า LLM และ Prompt Template (ตามข้อ 4.3)

In [ ]:
#ตั้งค่า Environment Variable
os.environ["GOOGLE_API_KEY"] = google_API_key

#สร้างโมเดล LLM
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",

    # แก้ไขตรงนี้: เรียกใช้ผ่าน os.environ โดยอ้างอิง "ชื่อตัวแปร" ที่เราตั้งไว้ข้างบน
    google_api_key=os.environ["GOOGLE_API_KEY"]
)

#สร้าง Prompt Template
rag_prompt_template = """
Context:
{context}

Question:
{question}

Instruction:
1. Answer in the Thai language.
2. Do not use outside knowledge or hallucinate information.
3. If the context is raw JSON data. Your task is to interpret this data and convert it into a natural, easy-to-read response.
4. Keep the answer professional and concise.
"""

prompt = PromptTemplate(
    template=rag_prompt_template,
    input_variables=["context", "question"]
)

เตรียมข้อมูลสำหรับ Vector Search

In [ ]:
#แปลง DataFrame แต่ละแถวให้เป็นข้อความ 1 ประโยค
#ตัวอย่าง: "Manchester United playsIn Premier League"
documents = [
    f"{row['Subject']} {row['Predicate']} {row['Object']}"
    for index, row in df.iterrows()
]

#สร้าง Embedding Model
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## Vector DB only (FAISS) (ข้อ 4.2.1)

In [ ]:
from langchain.chains import RetrievalQA

vectorstore_faiss = FAISS.from_texts(documents, embeddings)

#สร้าง Retriever (กำหนด k=5 ตามที่คุณต้องการก่อนหน้า)
retriever_faiss = vectorstore_faiss.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 5}
)

#สร้าง Chain พร้อมสั่งให้คืนค่า Source Documents
rag_1_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever_faiss,
    chain_type_kwargs={"prompt": prompt},
    return_source_documents=True
)
#ทดสอบและแสดงผล
query = "Which team plays in Premier League?"
response = rag_1_chain.invoke({"query": query})

print("\n=== ข้อมูล Context ที่ดึงมาใช้ (Top-5) ===")
for i, doc in enumerate(response['source_documents']):
    print(f"Docs #{i+1}: {doc.page_content}")

print(f"\nFinal Answer: {response['result']}")


=== ข้อมูล Context ที่ดึงมาใช้ (Top-5) ===
Docs #1: Manchester City playsIn Premier League
Docs #2: Chelsea playsIn Premier League
Docs #3: Leeds United playsIn Premier League
Docs #4: Liverpool playsIn Premier League
Docs #5: Newcastle United playsIn Premier League

Final Answer: ทีมที่เล่นในพรีเมียร์ลีก ได้แก่:
*   แมนเชสเตอร์ ซิตี้
*   เชลซี
*   ลีดส์ ยูไนเต็ด
*   ลิเวอร์พูล
*   นิวคาสเซิล ยูไนเต็ด


## Graph DB (Neo4j Relationships) (ข้อ 4.2.2)

In [ ]:
from langchain_neo4j import GraphCypherQAChain
from langchain_neo4j import Neo4jVector
from langchain_neo4j import Neo4jGraph

#เชื่อมต่อ Graph สำหรับ LangChain
graph = Neo4jGraph(url=uri, username=user, password=password)

#สร้าง Chain แบบ Graph Text-to-Cypher
rag_2_chain = GraphCypherQAChain.from_llm(
    llm = llm,
    graph = graph,
    verbose = True,            # ให้โชว์ Cypher ที่มันเขียนให้เราดู
    qa_prompt = prompt,        # ใช้ Prompt ภาษาไทยที่เราเตรียมไว้
    allow_dangerous_requests = True,
    top_k = 5
)

#ทดสอบ ถามหาความสัมพันธ์ที่ต้อง Join ข้อมูล
print("=== RAG 2 (Graph Pattern Match) ===")
query_rag2 = "Liverpool plays in which league and who is their rival?"
response_rag2 = rag_2_chain.invoke({"query": query_rag2})

print(f"\nFinal Answer: {response_rag2['result']}")

=== RAG 2 (Graph Pattern Match) ===


> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (liverpool:Entity {name: 'Liverpool'})-[:playsIn]->(league:Entity),
      (liverpool)-[:hasRival]->(rival:Entity)
RETURN league.name, rival.name

Full Context:
[{'league.name': 'Premier League', 'rival.name': 'Everton'}]

> Finished chain.

Final Answer: ลิเวอร์พูลเล่นในพรีเมียร์ลีก และคู่แข่งของพวกเขาคือเอฟเวอร์ตัน


## Knowledge Graph (Node + Relationship + Attribute) (ข้อ 4.2.3)

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

def run_hybrid_rag(user_query):
    print(f"1️ [User Query]: {user_query}")

    #Vector Grounding หา Node ตั้งต้น
    #ใช้ FAISS ที่เราทำไว้ใน RAG 1 มาช่วยหา "ตัวตน" ที่ใกล้เคียงที่สุด
    docs = vectorstore_faiss.similarity_search(user_query, k = 5)

    retrieved_text = docs[0].page_content
    print(f"2️ [Vector Found]: {retrieved_text}")

    #สร้าง Prompt สั้นๆ ให้ LLM ช่วยดึงชื่อ Subject ออกมา
    extraction_prompt = PromptTemplate(
        input_variables=["text"],
        template="Extract the main entity name (Subject) from this text: '{text}'. Return ONLY the name."
    )
    extractor_chain = extraction_prompt | llm | StrOutputParser()
    entity_name = extractor_chain.invoke({"text": retrieved_text}).strip()
    print(f"3️ [Entity Grounded]: {entity_name}")

    #Graph Traversal
    #เอาชื่อที่ถูกต้อง entity_name ไปรัน Cypher เพื่อดึงข้อมูลรอบตัว 1-hop
    cypher_query = """
    MATCH (n:Entity {name: $name})-[r]-(m)
    RETURN n.name AS Subject, type(r) AS Relationship, m.name AS Object
    """

    #รันคำสั่งใน Neo4j
    graph_data = graph.query(cypher_query, params={"name": entity_name})
    print(f"4️ [Graph Traversal]: Found {len(graph_data)} relationships")

    #Generate Answer
    #เอาผลลัพธ์จาก Graph ไปให้ LLM ตอบคำถาม
    context_str = str(graph_data)

    final_prompt = f"""
    Context from Graph Database:
    {context_str}

    User Question: {user_query}

    Answer in Thai (3 sentences):
    """

    final_answer = llm.invoke(final_prompt).content
    return final_answer

#ทดสอบ RAG 3 Hybrid
print("\n=== RAG 3 (Hybrid: Vector Grounding + Graph Traversal) ===")
#ลองแกล้งพิมพ์ผิด หรือถามกว้างๆ ให้ Vector ช่วยจับ
result = run_hybrid_rag("Tell me about Man U rivals")
print(f"\n Final Answer: {result}")


=== RAG 3 (Hybrid: Vector Grounding + Graph Traversal) ===
1️ [User Query]: Tell me about Man U rivals
2️ [Vector Found]: Manchester United hasNickname Red Devils
3️ [Entity Grounded]: Manchester United
4️ [Graph Traversal]: Found 12 relationships

 Final Answer: จากข้อมูล แมนเชสเตอร์ ยูไนเต็ดมีคู่ปรับสำคัญคือ ลิเวอร์พูล และ แมนเชสเตอร์ ซิตี้
ทั้งสองทีมนี้ถือเป็นคู่แข่งที่ดุเดือดและมีประวัติศาสตร์ยาวนานในวงการฟุตบอลอังกฤษ
การแข่งขันระหว่างแมนเชสเตอร์ ยูไนเต็ดกับทีมเหล่านี้มักจะเต็มไปด้วยความเข้มข้นและน่าติดตามเสมอ
